In [154]:
import datetime
import os, sys
import numpy as np
import matplotlib.pyplot as plt

import casadi as cas

##### For viewing the videos in Jupyter Notebook
import io
import base64
from IPython.display import HTML

# from ..</src> import car_plotting
# from .import src.car_plotting
PROJECT_PATH = '/home/nbuckman/Dropbox (MIT)/DRL/2020_01_cooperative_mpc/mpc-multiple-vehicles/'
sys.path.append(PROJECT_PATH)
import src.MPC_Casadi as mpc
import src.car_plotting as cplot
import src.TrafficWorld as tw
np.set_printoptions(precision=2)
import src.IterativeBestResponseMPCMultiple as mibr

In [129]:
list_of_subdirectory_names = [
"20200227_133704less_kxdotlarger",
# "20200227_165619constantV",
# "20200227_170553constantHighSlack",
]
for subdir_name in list_of_subdirectory_names:    
    folder = "results/" + subdir_name + "/"
    ################ Analyze Results

    rounds_ibr = 225
    n_other_cars = 4
    N = 50

    SAVE = True
    PLOT = False

    # ax_xmax = ax_xma[4]


    n_full_rounds = 0  # rounods that the ambulance planned 
    n_all_rounds = 0

    all_xamb = np.zeros((6, N+1, rounds_ibr))
    all_uamb = np.zeros((2, N, rounds_ibr))
    all_xcost = np.zeros((3, rounds_ibr))


    all_other_x = [np.zeros((6, N+1, rounds_ibr)) for i in range(n_other_cars)]
    all_other_u = [np.zeros((2, N, rounds_ibr)) for i in range(n_other_cars)]
    all_other_cost = [np.zeros((3, rounds_ibr)) for i in range(n_other_cars)]
    for amb_ibr_i in range(rounds_ibr):
        if (amb_ibr_i % (n_other_cars + 1) == 1) and amb_ibr_i>1:
            ibr_prefix =  '%03d'%amb_ibr_i
            try:
                xamb, uamb, xamb_des, xothers, uothers, xothers_des = mibr.load_state(folder + "data/" + ibr_prefix, n_other_cars)
                all_xamb[:,:,n_full_rounds] = xamb
                all_uamb[:,:,n_full_rounds] = uamb

                for i in range(n_other_cars):
                    all_other_x[i][:,:,n_full_rounds] = xothers[i] 
                    all_other_u[i][:,:,n_full_rounds] = uothers[i]
                n_full_rounds += 1
            except FileNotFoundError:
                # print("amb_ibr_i %d missing"%amb_ibr_i)
                pass
            n_all_rounds += 1

    ### Clip the extra dimension
    all_xamb = all_xamb[:,:,:n_full_rounds]
    all_uamb = all_uamb[:,:,:n_full_rounds]
    for i in range(n_other_cars):
        all_other_x[i] = all_other_x[i][:,:,:n_full_rounds]
        all_other_u[i] = all_other_u[i][:,:,:n_full_rounds]
    ibr_brounds_array = np.array(range(1, n_full_rounds +1))


    fig_trajectory, ax_trajectory = plt.subplots(1,1)
    ax_trajectory.set_title("Ambulance Trajectories")
    # fig_trajectory.set_figheight(fig_height)
    # fig_trajectory.set_figwidth(fig_width)
    fig_trajectory.set_size_inches((8,6))
    ax_trajectory.plot(all_xamb[0,:,:], all_xamb[1,:,:], '-o')
    ax_trajectory.set_xlabel("X [m]")
    ax_trajectory.set_ylabel("Y [m]")
    ud = all_uamb[0,:,:]
    if SAVE:
        fig_file_name = folder + 'plots/' + 'fig1_amb_trajectory.eps'
        fig_trajectory.savefig(fig_file_name, dpi=95, format='eps')
        print("Save to....", fig_file_name)

    ##########################################333333

    fig_uamb, ax_uamb = plt.subplots(3,1)
    # fig_uamb.set_figheight(fig_height)
    # fig_uamb.set_figwidth(4)
    fig_uamb.set_size_inches((8,8))
    fig_uamb.suptitle("Ambulance Control Input over IBR Iterations")
    ax_uamb[0].plot(ibr_brounds_array, np.sum(all_uamb[0,:,:] * all_uamb[0,:,:], axis=0), '-o')
    # ax_uamb[0].set_xlabel("IBR Iteration")
    ax_uamb[0].set_ylabel("$\sum u_{\delta}^2$")

    ax_uamb[1].plot(ibr_brounds_array, np.sum(all_uamb[1,:,:] * all_uamb[1,:,:], axis=0), '-o')
    # ax_uamb[1].set_xlabel("IBR Iteration")
    ax_uamb[1].set_ylabel("$\sum u_{v}^2$")

    ax_uamb[2].plot(ibr_brounds_array, np.sum(all_uamb[0,:,:] * all_uamb[0,:,:], axis=0) + np.sum(all_uamb[1,:,:] * all_uamb[1,:,:], axis=0), '-o')
    ax_uamb[2].set_xlabel("IBR Iteration")
    ax_uamb[2].set_ylabel("$\sum ||u||^2$")   

    if SAVE:
        fig_file_name = folder + 'plots/' + 'fig2_amb_ctrl_iterations.eps'
        fig_uamb.savefig(fig_file_name, dpi=95, format='eps')
        print("Save to....", fig_file_name)
        

    ##########################################################3
    fig_reluamb, ax_reluamb = plt.subplots(2,1)
    # fig_reluamb.set_figheight(fig_height)
    # fig_reluamb.set_figwidth(fig_width)
    fig_reluamb.set_size_inches((8,6))
    ax_reluamb[0].plot(ibr_brounds_array[1:], np.sum((all_uamb[0,:,1:]-all_uamb[0,:,0:-1])*(all_uamb[0,:,1:]-all_uamb[0,:,0:-1]), axis=0), '-o')
    # ax_reluamb[0].set_xlabel("IBR Iteration")
    ax_reluamb[0].set_ylabel("$\sum (u_{v\delta,t}-u_{\delta,t-1})^2$")

    ax_reluamb[1].plot(ibr_brounds_array[1:], np.sum((all_uamb[1,:,1:]-all_uamb[1,:,0:-1])*(all_uamb[1,:,1:]-all_uamb[1,:,0:-1]), axis=0), '-o')
    ax_reluamb[1].set_xlabel("IBR Iteration")
    ax_reluamb[1].set_ylabel("$\sum (u_{v,t}-u_{v,t-1})^2$")
    fig_reluamb.suptitle("Change in Ambulance Control Input over IBR Iterations")

    if SAVE:
        fig_file_name = folder + 'plots/' + 'fig3_change_amb_ctrl_iterations.eps'
        fig_reluamb.savefig(fig_file_name, dpi=95, format='eps')
        print("Save to....", fig_file_name)


    fig_xfinal, ax_xfinal = plt.subplots(2,1)
    fig_xfinal.suptitle("Final Ambulance State Over Iterations")
    fig_xfinal.set_size_inches((8,6))
    # fig_xfinal.set_figheight(fig_height)
    # fig_xfinal.set_figwidth(fig_width)
    ax_xfinal[0].plot(ibr_brounds_array, all_xamb[0,-1,:], '-o')
    # ax_reluamb[0].set_xlabel("IBR Iteration")
    ax_xfinal[0].set_ylabel("$x_{final}$")

    ax_xfinal[1].plot(ibr_brounds_array, all_xamb[2,-1,:], '-o')
    ax_xfinal[1].set_xlabel("IBR Iteration")
    ax_xfinal[1].set_ylabel(r"$\Theta_{final}$")
    if SAVE:
        fig_file_name = folder + 'plots/' + 'fig4_iterations_ambperformance.eps'
        fig_xfinal.savefig(fig_file_name, dpi=95, format='eps')
    print("Save to....", fig_file_name)

    ###################### NOW PLOTTING THE OTHER VEHICLES #########################3

    fig_xfinal_all, ax_xfinal_all = plt.subplots(3,1)
    fig_xfinal_all.suptitle("Comparing Distance Travel for the Vehicles")
    fig_xfinal_all.set_size_inches((8,8))
    # fig_xfinal_all.set_figheight(fig_height)
    # fig_xfinal_all.set_figwidth(fig_width)
    ax_xfinal_all[0].bar(range(n_other_cars + 1), [all_xamb[0, -1, -1] - all_xamb[0, 0, -1]] + [all_other_x[i][0,-1,-1] - all_other_x[i][0,0,-1] for i in range(n_other_cars)])
    ax_xfinal_all[0].set_ylabel("Horizontal Displacement $\Delta x$")
    ax_xfinal_all[0].set_xticks(range(n_other_cars + 1))
    ax_xfinal_all[0].set_xticklabels(["A"] + [str(i) for i in range(1, n_other_cars+1)])

    ax_xfinal_all[1].bar(range(n_other_cars + 1),  [all_xamb[-1, -1, -1] - all_xamb[-1, 0, -1]] + [all_other_x[i][-1,-1,-1] - all_other_x[i][-1,0,-1] for i in range(n_other_cars)])
    ax_xfinal_all[1].set_ylabel("Total Distance $s_f - s_i$")
    ax_xfinal_all[1].set_xticks(range(n_other_cars + 1))
    ax_xfinal_all[1].set_xticklabels(["A"] + [str(i) for i in range(1, n_other_cars+1)])

    ax_xfinal_all[2].bar(range(n_other_cars + 1),  [np.sum(all_xamb[2,:,-1]*all_xamb[2,:,-1])] +  [np.sum(all_other_x[i][2,:,-1]*all_other_x[i][2,:,-1]) for i in range(n_other_cars)])
    ax_xfinal_all[2].set_ylabel("Angular Deviation $\sum_{t} \Theta_t^2$")
    ax_xfinal_all[2].set_xticks(range(n_other_cars + 1))
    ax_xfinal_all[2].set_xticklabels(["A"] + [str(i) for i in range(1, n_other_cars+1)])

    if SAVE:
        fig_file_name = folder + 'plots/' + 'fig5_vehicles_comparison.eps'
        fig_xfinal_all.savefig(fig_file_name, dpi=95, format='eps')
        print("Save to....", fig_file_name)


    if PLOT:
        plt.show()

In [133]:
nonresponse_u_list[2]

array([[-0.01,  0.  ,  0.  , -0.01, -0.01, -0.03, -0.02,  0.01,  0.  ,
        -0.  , -0.02, -0.01,  0.02,  0.01, -0.03, -0.01,  0.  ,  0.03,
        -0.  , -0.02, -0.01, -0.03, -0.02, -0.02, -0.  ,  0.01, -0.01,
         0.02, -0.01, -0.02,  0.01,  0.03, -0.01, -0.  , -0.02, -0.  ,
         0.03,  0.02,  0.  ,  0.01,  0.02,  0.03, -0.  , -0.03,  0.01,
         0.03, -0.  ,  0.02, -0.02,  0.01],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

In [134]:
try:
    bri.solve(None, nonresponse_u_list)

    x1, u1, x1_des, _, _, _, other_x, other_u, other_des = bri.get_solution()
    xothers = other_x # initialize the x values of the other vehicles
    uothers = other_u

    print("n_round %d i %02d Cost %.02f Slack %.02f "%(n_round, i, bri.solution.value(bri.total_svo_cost), bri.solution.value(bri.slack_cost)))
    print("Dir:", folder)
    if bri.solution.value(bri.slack_cost) <= min_slack:
        uamb = u1
        xamb = x1
#         all_other_u[i] = u1
        file_name = folder + "data/"+'%03d'%ibr_sub_it
        mibr.save_state(file_name, x1, u1, x1_des, other_x, other_u, other_des)
        mibr.save_costs(file_name, bri)          
    else: 
        print("Slack too large")
except RuntimeError:
    print("Max Iterations or Infeasible")
    runtimeerrors += 1                
ibr_sub_it +=1

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 186.65ms ( 65.49us) 185.28ms ( 65.01us)      2850
       nlp_g  |   5.09 s (  1.79ms)   5.06 s (  1.77ms)      2850
    nlp_grad  |   6.10ms (  6.10ms)   6.09ms (  6.09ms)         1
  nlp_grad_f  | 413.66ms (156.27us) 411.35ms (155.40us)      2647
  nlp_hess_l  |  62.82 s ( 22.46ms)  62.39 s ( 22.31ms)      2797
   nlp_jac_g  |  30.78 s ( 10.96ms)  30.57 s ( 10.88ms)      2809
       total  | 155.19 s (155.19 s) 154.12 s (154.12 s)         1
n_round 0 i 03 Cost 145756.92 Slack 14.52 
Dir: results/20200227_112411allsame/


In [153]:
print(int((6-1)/5))
print(int((11-1)/5))
print(int((16-1)/5))

1
2
3


2.0

In [ ]:
for i in range(len(all_other_MPC)):
    response_MPC = all_other_MPC[i]
    response_x0 = all_other_x0[i]

    nonresponse_MPC_list = all_other_MPC[:i] + all_other_MPC[i+1:]
    nonresponse_x0_list = all_other_x0[:i] + all_other_x0[i+1:]

    # all_other_u changes over time
    nonresponse_u_list = all_other_u[:i] + all_other_u[i+1:]

    bri = mibr.IterativeBestResponseMPCMultiple(response_MPC, amb_MPC, nonresponse_MPC_list )
    bri.k_slack = 9999

    bri.generate_optimization(N, T, response_x0, x0_amb, nonresponse_x0_list,  1, slack=True)

    try:
        if WARM:
            bri.opti.set_initial(bri.x_opt, xothers[i])
            bri.opti.set_initial(bri.u_opt, uothers[i])

        bri.solve(uamb, nonresponse_u_list)
        x1, u1, x1_des, xamb, uamb, xamb_des, other_x, other_u, other_des = bri.get_solution()
        print("n_round %d  i %02d Cost %.02f Slack %.02f "%(n_round, i, bri.solution.value(bri.total_svo_cost), bri.solution.value(bri.slack_cost)))
        print("Dir:", folder)

        if bri.solution.value(bri.slack_cost) <= min_slack:
            # Update the responder
            all_other_u[i] = u1

            #for saving
            xothers = other_x[:i] + [x1] + other_x[i:]
            uothers = other_u[:i] + [u1] + other_u[i:]
            xothers_des = other_des[:i] + [x1_des] + other_des[i:]

            file_name = folder + "data/"+'%03d'%ibr_sub_it
            mibr.save_state(file_name, xamb, xamb, xamb_des, xothers, uothers, xothers_des)
            mibr.save_costs(file_name, bri)
        else: 
            print("Slack too large")    
        ibr_sub_it+=1
    except RuntimeError:
        print("Max Iterations or Infeasible")
        runtimeerrors += 1         


# if WARM:
#     br1.opti.set_initial(br1.u_opt, u1) 
#     br1.opti.set_initial(br1.x_opt, x1)                 
#     br1.opti.set_initial(br1.x_desired, x1_des)                 
